In [1]:
pip install kagglehub

  Obtaining dependency information for kagglehub from https://files.pythonhosted.org/packages/49/bf/c2a24567bb6bd80c1fe7cb2ed1a332666476f69c313256aff96094bef93e/kagglehub-0.3.12-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
   ---------------------------------------- 68.0/68.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import kagglehub
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [ ]:
# Download latest version
path = kagglehub.dataset_download("christang0002/davis-and-kiba")

print("Path to dataset files:", path)

c:\Users\NongNam\Documents\AI_Builder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 55.5M/55.5M [00:04<00:00, 12.9MB/s]

Extracting files...


Path to dataset files: C:\Users\NongNam\.cache\kagglehub\datasets\christang0002\davis-and-kiba\versions\3


In [8]:
df = pd.read_csv('davis.txt', header=None, sep=' ')

df.columns  = ['drug_id', 'prot_id', 'drug_smile', 'prot_seq', 'label']
df.head()

,drug_id,prot_id,drug_smile,prot_seq,label
0,11314340,AAK1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.366532
1,11314340,ABL1(E255K),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
2,11314340,ABL1(F317I),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
3,11314340,ABL1(F317I)p,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
4,11314340,ABL1(F317L),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000


In [9]:
df = df.drop(columns=['drug_id', 'prot_id'])
df = df.rename(columns={'label' : 'Kd','drug_smile' : 'SMILES', 'prot_seq' : 'target_sequence'})

In [10]:
df.head()

,SMILES,target_sequence,Kd
0,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.366532
1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
2,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
3,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000
4,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000


In [15]:
df.isna().sum()

SMILES             0
target_sequence    0
Kd                 0
smiles_encoded     0
protein_encoded    0
dtype: int64

In [11]:
# 3. Encode SMILES
smiles_chars = sorted(set(''.join(df['SMILES'])))
char_to_int_smiles = {c: i + 1 for i, c in enumerate(smiles_chars)}

def encode_smiles(smiles):
    return [char_to_int_smiles[c] for c in smiles]

df['smiles_encoded'] = df['SMILES'].apply(encode_smiles)

# 4. Encode protein sequences
aa_list = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: i + 1 for i, aa in enumerate(aa_list)}

def encode_protein(seq):
    return [aa_to_int.get(aa, 0) for aa in seq]

df['protein_encoded'] = df['target_sequence'].apply(encode_protein)

In [12]:
max_len_smiles = 100
max_len_protein = 1000

X_smiles = pad_sequences(df['smiles_encoded'], maxlen=max_len_smiles, padding='post')
X_protein = pad_sequences(df['protein_encoded'], maxlen=max_len_protein, padding='post')
y = df['Kd'].values

In [13]:
# First split into train + test
X_s_train, X_s_test, X_p_train, X_p_test, y_train_val, y_test = train_test_split(
    X_smiles, X_protein, y, test_size=0.2, random_state=42
)

# Then split train into train + val
X_s_train, X_s_val, X_p_train, X_p_val, y_train, y_val = train_test_split(
    X_s_train, X_p_train, y_train_val, test_size=0.2, random_state=42
)

In [14]:
len(X_s_train), len(X_s_val), len(X_s_test)

(19235, 4809, 6012)

In [16]:
model = load_model("model_epoch_94.h5")

# Recompile before training again
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Fine-tune the model
model.fit(
    [X_s_train, X_p_train], y_train,
    validation_data=([X_s_val, X_p_val], y_val),
    epochs=100, batch_size=32
)

Epoch 1/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 204s 333ms/step - loss: 0.6853 - mse: 0.6853 - val_loss: 0.5550 - val_mse: 0.5550
Epoch 2/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 191s 317ms/step - loss: 0.4813 - mse: 0.4813 - val_loss: 0.4180 - val_mse: 0.4180
Epoch 3/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 188s 313ms/step - loss: 0.4434 - mse: 0.4434 - val_loss: 0.3851 - val_mse: 0.3851
Epoch 4/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 182s 303ms/step - loss: 0.4374 - mse: 0.4374 - val_loss: 0.3930 - val_mse: 0.3930
Epoch 5/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 195s 323ms/step - loss: 0.4409 - mse: 0.4409 - val_loss: 0.3789 - val_mse: 0.3789
Epoch 6/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 208s 346ms/step - loss: 0.3981 - mse: 0.3981 - val_loss: 0.3697 - val_mse: 0.3697
Epoch 7/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 190s 316ms/step - loss: 0.3922 - mse: 0.3922 - val_loss: 0.3497 - val_mse: 0.3497
Epoch 8/10
602/602 ━━━━━━━━━━━━━━━━━━━━ 194s 322ms/step - loss: 0.3706 - mse: 0.3706 - val_loss: 0.3660 - val_mse: 0.3660
Epoch 9/10
602/602 ━━━━━

In [19]:
loss, mse = model.evaluate([X_s_test, X_p_test], y_test)
print(f"Test MSE: {mse:.4f}")

188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - loss: 0.3069 - mse: 0.3069
Test MSE: 0.3104


In [ ]:
model.save("fine_tuned_model.h5")